In [6]:
import tensorflow as tf
import glob
import matplotlib.pyplot as plt
import numpy as np
import os
import zipfile
import cv2
from matplotlib import image
import sys

In [7]:
# Names of models to load
generator = "COVID_CT_GAN_Generator.h5"
discriminator = "COVID_CT_GAN_Discriminator.h5"
gan = "COVID_CT_GAN_GAN.h5"

# Name of dataset zip file without .zip extension
Dataset = "CT_COVID"

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
# filepath = "/content/drive/My Drive/Colab Notebooks/COVID19-GAN"
filepath = "/content/drive/My Drive/Colab Notebooks/COVID_GAN_Saved_Models"
destination = "/drive/Data"

generator = tf.keras.models.load_model(filepath+"/generator_final_5000.h5")
discriminator = tf.keras.models.load_model(filepath+"/discriminator_final_5000.h5")
gan = tf.keras.models.load_model(filepath+"/gan_final_5000.h5")

In [10]:
# sys.path.append(filepath)
sys.path.insert(0, filepath)
import imp
import interpolate
imp.reload(interpolate)
from interpolate import model_progression

In [ ]:
# Will unzip the files so that you can see them..
with zipfile.ZipFile(filepath+"/"+Dataset+".zip","r") as z:
    z.extractall(destination)

from subprocess import check_output
os.chdir(destination + "/" + Dataset)

path = "*.*"
data = []
for file in glob.glob(path):
  if(file.find("image_at_epoch") == -1):
    img_data = image.imread(file, 0)
    data.append(img_data)  
data = np.array(data)

from skimage.color import rgb2gray
WIDTH=128
HEIGHT=128
standardized_data = []
for img in data:
  img = rgb2gray(img)
  img2 = np.flip(img, axis=1) ## ADD FLIPPED IMAGE FOR LARGER DATA SET
  standardized_data.append(cv2.resize(img, (WIDTH,HEIGHT), interpolation=cv2.INTER_CUBIC).astype('float32'))
  standardized_data.append(cv2.resize(img2, (WIDTH,HEIGHT), interpolation=cv2.INTER_CUBIC).astype('float32'))
train_images = np.array(standardized_data)
train_images = train_images.reshape(train_images.shape[0], 128,128,1).astype('float32')
print(train_images.shape)

train_images = (train_images - 127.5) / 127.5  # Normalize the images to [-1, 1]
#os.system("rm -r \""+destination+"\"")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


In [ ]:
noise = tf.random.normal([1, 100])
generated_image0 = generator.predict(noise)
plt.imshow(generated_image0[0, :, :, 0], cmap='gray')
plt.show()

In [ ]:
f = plt.figure()
noise_dim = 100
for i in range(1,26):
  f.add_subplot(5,5, i)
  noise = np.random.randn(noise_dim)
  noise = noise.reshape(1, noise_dim)
  generated_image0 = generator.predict(noise)
  plt.imshow(generated_image0[0, :, :, 0], cmap='gray')
  plt.xticks([])
  plt.yticks([])
plt.show()

In [ ]:
noise =[[0,0,0,0,0,0,0,0,0,0,
         0,0,0,0,0,0,0,0,0,0,
         0,0,0,0,0,0,0,0,0,0,
         0,0,0,0,0,0,0,0,0,0,
         0,0,0,0,0,0,0,0,0,0,
         0,0,0,0,0,0,0,0,0,0,
         0,0,0,0,0,0,0,0,0,0,
         0,0,0,0,0,0,0,0,0,0,
         0,0,0,0,0,0,0,0,0,0,
         0,0,0,0,0,0,0,0,0,0,]]
generated_image0 = generator.predict(noise)

plt.imshow(generated_image0[0, :, :, 0], cmap='gray')
plt.show()


In [ ]:
start_noise = tf.random.normal([1, 100])
generated_image1 = generator.predict(start_noise)

end_noise = tf.random.normal([1, 100])
generated_image2 = generator.predict(end_noise)

f = plt.figure()
f.add_subplot(1, 2, 1)
plt.title('Start Image')
plt.imshow(generated_image1[0,:,:,0], cmap='gray')
f.add_subplot(1, 2, 2)
plt.title('End Image')
plt.imshow(generated_image2[0,:,:,0], cmap='gray')
plt.show()

In [ ]:
model_progression(generator, start=start_noise, end=end_noise, steps=6, display=6)